In [2]:
import pandas as pd
import os
pd.set_option('display.float_format', '{:.3f}'.format)

In [24]:
def write_df_in_latex(df, label, short_caption, long_caption):
    print("\\begin{table}[h]")
    print("\t\\centering")
    print("\t\\caption[" + short_caption +"]{"+ long_caption + "}\n\t\\tiny")

    print(df.to_latex(index=False))

    print("\t\\label{tab:" + label + "}\n\\end{table}")

# Metrics taken in account

## Accuracy

Accuracy is calculated as:

$$
\text{Accuracy} = \frac{\text{True Positive + True Negative}}{\text{True Positive + False Positive + True Negative + False Negative}}
$$


## Precision

The fraction of produced detections which are true positives.

$$
\text{Precision} = \frac{\text{True Positive}}{\text{True Positive + False Positive}} = \frac{\text{True Positive}}{\text{Total Number of Prediction}}
$$


## Recall

The fraction of groundtruth boxes in the data that matched to some produced detection.

$$
\text{Recall} = \frac{\text{True Positive}}{\text{True Positive + False Negative}}
$$

## F1 score

F1 calcula el balance entre precision y recall. Si el F1 es alto, precision y recall son altos.

The F1 Score is calculated as:

$$
\text{F1 Score} = \frac{2 \times (\text{Precision} \times \text{Recall})}{\text{Precision} + \text{Recall}}
$$

# The problem

We are dealing with a weapon detection issue. It is more important to avoid false negatives than false positives. It is better not to miss a weapon detection, at the cost of detecting more weapons with less accuracy. For this reason, a low threshold will also be used.

Given the context of weapon detection and the emphasis on avoiding false negatives (missing weapon detections) at the expense of potentially having more false positives (incorrect weapon detections), **recall** is more important than precision in this scenario.

Recall, also known as sensitivity or true positive rate, measures the proportion of actual positive cases (weapons) that were correctly identified by the model. In the context of your problem, a higher recall means that the model is effectively capturing a larger portion of actual weapons, minimizing the risk of missing any potentially dangerous objects.

While precision (positive predictive value) is also significant, it prioritizes the accuracy of positive predictions among all predicted positives. In your case, a focus on precision might lead to being overly cautious and producing fewer false positives, but it could also result in missing actual weapon detections, which is a more critical concern in a weapon detection scenario.

A higher **recall** rate is crucial for ensuring that potential weapons are not overlooked, even if it means accepting a higher number of false positives.

# Evaluating models vs small dataset
## Training 50 epochs

In [79]:
OUTPUT_DIR = "./output_csvs"
for filename in os.listdir(OUTPUT_DIR):
    if filename.endswith(".csv"):
        print(filename)

eval_trained_on_dataset_v1_task_test_ds_original.csv
eval_trained_on_dataset_v2_task_test_ds_rc_complete.csv
eval_trained_on_dataset_v1_task_val_ds_original.csv
eval_trained_on_dataset_v2_task_test_ds_original.csv
eval_trained_on_dataset_v1_task_test_ds_rc.csv
eval_trained_on_dataset_v2_task_test_ds_rc.csv
eval_trained_on_dataset_v2_task_val_ds_original.csv


In [21]:
# Define the new column names mapping
new_column_names = {
    'model': 'Model',
    'imgsz': 'imgsz',
    'transfer_learning': 'TL',
    'lr0': 'LR',
    'optimizer': 'Optim.',
    'all_P': 'P',
    'all_R': 'R',
    'all_F1': 'F1-score',
    'all_mAP@.5': 'mAP@.5',
    'all_mAP@.5:.95': 'mAP@.5:.95'
}

In [55]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v1_task_test_ds_original.csv")
# df.set_index('model_key', inplace=True)

columns_of_interest = ['model','imgsz','transfer_learning', 'lr0', 'optimizer', 'all_P', 'all_R',
                       'all_F1', 'all_mAP@.5', 'all_mAP@.5:.95']

df_50 = df[(df['epochs'] == 50)][columns_of_interest].sort_values(by=['all_P']).rename(columns=new_column_names)
df_50

,Model,imgsz,TL,LR,Optim.,P,R,F1-score,mAP@.5,mAP@.5:.95
1,yolov5,800,no,0.010,SGD,0.737,0.625,0.338,0.701,0.358
4,yolov5,640,yolov5s,0.010,SGD,0.773,0.712,0.371,0.764,0.429
6,yolov5,800,yolov5s,0.010,SGD,0.796,0.716,0.377,0.755,0.428
9,yolov7,640,yolov7training,0.001,SGD,0.839,0.902,0.435,0.907,0.626
12,yolov8,800,no,0.010,SGD,0.842,0.744,0.395,0.841,0.579
18,yolov8,800,yolov8s,0.010,SGD,0.864,0.776,0.409,0.852,0.583
16,yolov8,640,yolov8s,0.001,Adam,0.868,0.782,0.411,0.857,0.595


In [26]:
short_caption = "Experimentos con 50 épocas"

long_caption = """Tabla comparativa de experimentos realizados en 50 épocas con respecto al modelo, 
tamaño de la imagen (imgsz), transfer learning (TL), precisión (P), recall (R), F1-score, 
mAP@.5, mAP@.5:.95, tasa de aprendizaje (LR) y optimizador utilizado."""

write_df_in_latex(df_50,"50epochs",short_caption,long_caption)

\begin{table}[h]
	\centering
	\caption[Experimentos con 50 épocas]{Tabla comparativa de experimentos realizados en 50 épocas con respecto al modelo, 
tamaño de la imagen (imgsz), transfer learning (TL), precisión (P), recall (R), F1-score, 
mAP@.5, mAP@.5:.95, tasa de aprendizaje (LR) y optimizador utilizado.}
	\tiny
\begin{tabular}{lrlrlrrrrr}
\toprule
 Model &  imgsz &             TL &    LR & Optim. &     P &     R &  F1-score &  mAP@.5 &  mAP@.5:.95 \\
\midrule
yolov5 &    800 &             no & 0.010 &    SGD & 0.737 & 0.625 &     0.338 &   0.701 &       0.358 \\
yolov5 &    640 &        yolov5s & 0.010 &    SGD & 0.773 & 0.712 &     0.371 &   0.764 &       0.429 \\
yolov5 &    800 &        yolov5s & 0.010 &    SGD & 0.796 & 0.716 &     0.377 &   0.755 &       0.428 \\
yolov7 &    640 & yolov7training & 0.001 &    SGD & 0.839 & 0.902 &     0.435 &   0.907 &       0.626 \\
yolov8 &    800 &             no & 0.010 &    SGD & 0.842 & 0.744 &     0.395 &   0.841 &       0.579 \\
yolov

### Initial conclusions

The experiments were on yolov5, yolov7 and yolov8, and are evaluated on different image sizes, number of epochs, transfer learning and learning rate. The first experiments were trained in 50 epochs. 

Among the models, YOLOv8 generally performs better than YOLOv5 and YOLOv7 in terms of precision, but YOLOv7 performs better in recall and in mAP.

#### Transfer learning
Some experiments use transfer learning by starting with pre-trained weights ('yolov5s.pt', 'yolov7_training.pt', 'yolov8s.pt'). Transfer learning seems to generally improve performance in terms of precision, recall, and F1-score.

#### Loss function and learning rate

A smaller learning rate leads to more optimal models but takes longer to train. The ADAM optimizer seems to have better accuracy than the SGD optimizer.

## Training 100 epochs 

In [34]:
columns_of_interest

['model',
 'imgsz',
 'transfer_learning',
 'lr0',
 'optimizer',
 'all_P',
 'all_R',
 'all_F1',
 'all_mAP@.5',
 'all_mAP@.5:.95']

In [58]:
df_100 = df[["epochs"] + columns_of_interest].sort_values(by=['model', 'imgsz',
 'transfer_learning',
 'lr0',
 'optimizer', 'all_P']).rename(columns=new_column_names)

df_100 = df[["epochs"] + columns_of_interest].sort_values(by=['all_R']).rename(columns=new_column_names)
df_100

,epochs,Model,imgsz,TL,LR,Optim.,P,R,F1-score,mAP@.5,mAP@.5:.95
2,100,yolov5,640,yolov5s,0.001,SGD,0.692,0.603,0.322,0.665,0.333
1,50,yolov5,800,no,0.010,SGD,0.737,0.625,0.338,0.701,0.358
0,100,yolov5,800,no,0.010,SGD,0.793,0.689,0.369,0.747,0.422
4,50,yolov5,640,yolov5s,0.010,SGD,0.773,0.712,0.371,0.764,0.429
6,50,yolov5,800,yolov5s,0.010,SGD,0.796,0.716,0.377,0.755,0.428
12,50,yolov8,800,no,0.010,SGD,0.842,0.744,0.395,0.841,0.579
5,100,yolov5,800,yolov5s,0.010,SGD,0.854,0.745,0.398,0.807,0.490
3,100,yolov5,640,yolov5s,0.010,SGD,0.811,0.750,0.390,0.807,0.476
17,100,yolov8,800,yolov8s,0.010,SGD,0.877,0.756,0.406,0.844,0.612
13,100,yolov8,800,yolov8l,0.010,SGD,0.867,0.758,0.404,0.838,0.597


In [64]:
df_100 = df[(df['epochs'] == 100)][columns_of_interest].sort_values(by=['all_R']).rename(columns=new_column_names)
df_100

,Model,imgsz,TL,LR,Optim.,P,R,F1-score,mAP@.5,mAP@.5:.95
2,yolov5,640,yolov5s,0.001,SGD,0.692,0.603,0.322,0.665,0.333
0,yolov5,800,no,0.010,SGD,0.793,0.689,0.369,0.747,0.422
5,yolov5,800,yolov5s,0.010,SGD,0.854,0.745,0.398,0.807,0.490
3,yolov5,640,yolov5s,0.010,SGD,0.811,0.750,0.390,0.807,0.476
17,yolov8,800,yolov8s,0.010,SGD,0.877,0.756,0.406,0.844,0.612
13,yolov8,800,yolov8l,0.010,SGD,0.867,0.758,0.404,0.838,0.597
11,yolov8,800,no,0.010,SGD,0.871,0.776,0.410,0.846,0.599
15,yolov8,640,yolov8s,0.010,SGD,0.888,0.777,0.414,0.860,0.623
14,yolov8,640,yolov8s,0.001,Adam,0.852,0.796,0.412,0.849,0.616
10,yolov7,800,yolov7training,0.010,SGD,0.856,0.808,0.416,0.858,0.583


In [62]:
short_caption = "Experimentos con 100 épocas"

long_caption = """Tabla comparativa de experimentos realizados en 100 épocas con respecto al modelo, 
tamaño de la imagen (imgsz), transfer learning (TL), precisión (P), recall (R), F1-score, 
mAP@.5, mAP@.5:.95, tasa de aprendizaje (LR) y optimizador utilizado."""

write_df_in_latex(df_100,"100epochs",short_caption,long_caption)

\begin{table}[h]
	\centering
	\caption[Experimentos con 100 épocas]{Tabla comparativa de experimentos realizados en 100 épocas con respecto al modelo, 
tamaño de la imagen (imgsz), transfer learning (TL), precisión (P), recall (R), F1-score, 
mAP@.5, mAP@.5:.95, tasa de aprendizaje (LR) y optimizador utilizado.}
	\tiny
\begin{tabular}{lrlrlrrrrr}
\toprule
 Model &  imgsz &             TL &    LR & Optim. &     P &     R &  F1-score &  mAP@.5 &  mAP@.5:.95 \\
\midrule
yolov5 &    640 &        yolov5s & 0.001 &    SGD & 0.692 & 0.603 &     0.322 &   0.665 &       0.333 \\
yolov5 &    800 &             no & 0.010 &    SGD & 0.793 & 0.689 &     0.369 &   0.747 &       0.422 \\
yolov5 &    640 &        yolov5s & 0.010 &    SGD & 0.811 & 0.750 &     0.390 &   0.807 &       0.476 \\
yolov8 &    640 &        yolov8s & 0.001 &   Adam & 0.852 & 0.796 &     0.412 &   0.849 &       0.616 \\
yolov5 &    800 &        yolov5s & 0.010 &    SGD & 0.854 & 0.745 &     0.398 &   0.807 &       0.490 \\
yol

Increasing the number of training epochs from 50 to 100 seems to have improved the performance of most models across various metrics.

Higher epochs might allow the models to learn more complex patterns, leading to better results.

From the table, we can draw the following conclusions:

1. **Size Doesn't Always Mean Better:** The table suggests that having a bigger model does not necessarily guarantee better performance. For instance, the yolov8l model, which is larger than yolov8s, does not perform better.

2. **Impact of Image Size:** For every model, it is shown that having a 640 imgsz gives a better recall.

3. **Model Version:** Different versions of YOLO show differences in performance. Some versions might perform better in terms of certain metrics. In terms of recall, yolov7 performs better than yolov5 and yolov8. In terms of precision, the best model was achieved with yolov8.

4. **Transfer learning:** Across various configurations, using transfer learning consistently yields higher precision (P), recall (R), F1-score (F1), and mean Average Precision (mAP) metrics compared to non-transfer learning. This highlights the effectiveness of leveraging pre-trained models for better object detection performance.

5. **Precision and Recall Trade-off:** It's clear that there's a trade-off between precision and recall. Some models have higher precision but lower recall, while others have higher recall but lower precision.

Besides the metrics, the practical aspects like computational resources and inference speed will be considered choosing a model for deployment.

In [51]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v1_task_test_ds_rc.csv")

df[columns_of_interest].sort_values(by=['all_P'])

,model,epochs,imgsz,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
5,yolov8,50,640,yolov8s,0.00100,Adam,0.12400,0.15500,0.06889,0.06650,0.02570
1,yolov8,50,800,no,0.01000,SGD,0.13500,0.12900,0.06597,0.05420,0.02020
0,yolov8,100,800,no,0.01000,SGD,0.13700,0.11700,0.06311,0.04880,0.01980
2,yolov8,100,800,yolov8l,0.01000,SGD,0.15700,0.12700,0.07021,0.06150,0.02610
4,yolov8,100,640,yolov8s,0.01000,SGD,0.15900,0.10700,0.06396,0.04840,0.01990
7,yolov8,50,800,yolov8s,0.01000,SGD,0.21600,0.17600,0.09698,0.08590,0.02960
6,yolov8,100,800,yolov8s,0.01000,SGD,0.22200,0.11800,0.07705,0.07260,0.03070
3,yolov8,100,640,yolov8s,0.00100,Adam,0.32300,0.11000,0.08206,0.09760,0.03340


# Training models on a bigger dataset

Ideally, considering the previous evaluations, the optimal choice for training models on a larger dataset would have been yolov7. However, due to resource limitations on the training platform (Google Colab), specifically in terms of memory allocation, it proved unfeasible. Attempts to train the models on a bigger dataset resulted in memory overflows, abruptly terminating the training process before the model could be effectively trained... So, the next option was yolov8.

- image size: 640
- batch size: 16
- transfer learning: yolov8s
- lr: between 0.001 (with Adam) and 0.01 (with SGD)
- epochs: 100 and 300, to test if training longer gives better results

In [74]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v2_task_val_ds_original.csv")

columns_of_interest = ['model','epochs','imgsz','transfer_learning', 'lr0', 'optimizer', 'all_P', 'all_R',
                       'all_F1', 'all_mAP@.5', 'all_mAP@.5:.95']

df_v2 = df[columns_of_interest].sort_values(by=['epochs','all_P']).rename(columns=new_column_names)
# df_v2 = df[columns_of_interest].sort_values(by=['all_P']).rename(columns=new_column_names)
df_v2

,Model,epochs,imgsz,TL,LR,Optim.,P,R,F1-score,mAP@.5,mAP@.5:.95
8,yolov8,40,800,yolov8s,0.010,SGD,0.910,0.862,0.443,0.928,0.705
0,yolov8,50,640,yolov8l,0.001,Adam,0.916,0.831,0.436,0.909,0.704
4,yolov8,100,640,yolov8s,0.001,Adam,0.916,0.873,0.447,0.927,0.717
7,yolov8,100,800,yolov8s,0.010,SGD,0.922,0.883,0.451,0.938,0.728
3,yolov8,50,640,yolov8m,0.010,SGD,0.925,0.874,0.449,0.938,0.731
6,yolov8,50,640,yolov8s,0.001,Adam,0.925,0.836,0.439,0.915,0.705
2,yolov8,100,640,yolov8m,0.010,SGD,0.926,0.891,0.454,0.941,0.743
1,yolov8,50,640,yolov8l,0.010,SGD,0.927,0.880,0.451,0.944,0.733
9,yolov8,60,800,yolov8s,0.010,SGD,0.931,0.879,0.452,0.940,0.723
10,yolov8,80,800,yolov8s,0.010,SGD,0.931,0.879,0.452,0.938,0.727


In [70]:
short_caption = "Experimentos entrenados con el dataset más extenso."

long_caption = """Tabla de experimentos entrenados con el dataset más extenso. Comparación con respecto al modelo, 
tamaño de la imagen (imgsz), transfer learning (TL), precisión (P), recall (R), F1-score, 
mAP@.5, mAP@.5:.95, tasa de aprendizaje (LR) y optimizador utilizado."""

write_df_in_latex(df_v2,"dsv2",short_caption,long_caption)

\begin{table}[h]
	\centering
	\caption[Experimentos entrenados con el dataset más extenso.]{Tabla de experimentos entrenados con el dataset más extenso. Comparación con respecto al modelo, 
tamaño de la imagen (imgsz), transfer learning (TL), precisión (P), recall (R), F1-score, 
mAP@.5, mAP@.5:.95, tasa de aprendizaje (LR) y optimizador utilizado.}
	\tiny
\begin{tabular}{lrrlrlrrrrr}
\toprule
 Model &  epochs &  imgsz &      TL &    LR & Optim. &     P &     R &  F1-score &  mAP@.5 &  mAP@.5:.95 \\
\midrule
yolov8 &      40 &    800 & yolov8s & 0.010 &    SGD & 0.910 & 0.862 &     0.443 &   0.928 &       0.705 \\
yolov8 &      50 &    640 & yolov8l & 0.001 &   Adam & 0.916 & 0.831 &     0.436 &   0.909 &       0.704 \\
yolov8 &      50 &    640 & yolov8m & 0.010 &    SGD & 0.925 & 0.874 &     0.449 &   0.938 &       0.731 \\
yolov8 &      50 &    640 & yolov8s & 0.001 &   Adam & 0.925 & 0.836 &     0.439 &   0.915 &       0.705 \\
yolov8 &      50 &    640 & yolov8l & 0.010 &    SGD &

In [83]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v2_task_test_ds_rc_complete.csv")

df_rc = df[columns_of_interest].sort_values(by=['all_P']).rename(columns=new_column_names)
df_rc

,Model,epochs,imgsz,TL,LR,Optim.,P,R,F1-score,mAP@.5,mAP@.5:.95
6,yolov8,50,640,yolov8s,0.001,Adam,0.307,0.302,0.152,0.214,0.083
3,yolov8,50,640,yolov8m,0.010,SGD,0.331,0.345,0.169,0.285,0.114
0,yolov8,50,640,yolov8l,0.001,Adam,0.352,0.276,0.155,0.210,0.080
8,yolov8,40,800,yolov8s,0.010,SGD,0.372,0.327,0.174,0.268,0.110
4,yolov8,100,640,yolov8s,0.001,Adam,0.435,0.286,0.173,0.269,0.102
1,yolov8,50,640,yolov8l,0.010,SGD,0.481,0.347,0.202,0.309,0.130
5,yolov8,300,640,yolov8s,0.010,SGD,0.509,0.311,0.193,0.325,0.137
10,yolov8,80,800,yolov8s,0.010,SGD,0.560,0.302,0.196,0.310,0.123
2,yolov8,100,640,yolov8m,0.010,SGD,0.576,0.335,0.212,0.366,0.145
7,yolov8,100,800,yolov8s,0.010,SGD,0.578,0.302,0.198,0.314,0.124


me gustan los ultimos 3 (a revisar)

In [84]:
short_caption = "Validación sobre ``Randomized Clips''."

long_caption = """Validación sobre ``Randomized Clips''. Comparación con respecto al modelo, 
tamaño de la imagen (imgsz), transfer learning (TL), precisión (P), recall (R), F1-score, 
mAP@.5, mAP@.5:.95, tasa de aprendizaje (LR) y optimizador utilizado."""

write_df_in_latex(df_rc,"df_rc",short_caption,long_caption)

\begin{table}[h]
	\centering
	\caption[Validación sobre ``Randomized Clips''.]{Validación sobre ``Randomized Clips''. Comparación con respecto al modelo, 
tamaño de la imagen (imgsz), transfer learning (TL), precisión (P), recall (R), F1-score, 
mAP@.5, mAP@.5:.95, tasa de aprendizaje (LR) y optimizador utilizado.}
	\tiny
\begin{tabular}{lrrlrlrrrrr}
\toprule
 Model &  epochs &  imgsz &      TL &    LR & Optim. &     P &     R &  F1-score &  mAP@.5 &  mAP@.5:.95 \\
\midrule
yolov8 &      50 &    640 & yolov8s & 0.001 &   Adam & 0.307 & 0.302 &     0.152 &   0.214 &       0.083 \\
yolov8 &      50 &    640 & yolov8m & 0.010 &    SGD & 0.331 & 0.345 &     0.169 &   0.285 &       0.114 \\
yolov8 &      50 &    640 & yolov8l & 0.001 &   Adam & 0.352 & 0.276 &     0.155 &   0.210 &       0.080 \\
yolov8 &      40 &    800 & yolov8s & 0.010 &    SGD & 0.372 & 0.327 &     0.174 &   0.268 &       0.110 \\
yolov8 &     100 &    640 & yolov8s & 0.001 &   Adam & 0.435 & 0.286 &     0.173 &   0.26

In [82]:
df[columns_of_interest].sort_values(by=['all_F1']).rename(columns=new_column_names)

,Model,epochs,imgsz,TL,LR,Optim.,P,R,F1-score,mAP@.5,mAP@.5:.95
6,yolov8,50,640,yolov8s,0.001,Adam,0.307,0.302,0.152,0.214,0.083
0,yolov8,50,640,yolov8l,0.001,Adam,0.352,0.276,0.155,0.210,0.080
3,yolov8,50,640,yolov8m,0.010,SGD,0.331,0.345,0.169,0.285,0.114
4,yolov8,100,640,yolov8s,0.001,Adam,0.435,0.286,0.173,0.269,0.102
8,yolov8,40,800,yolov8s,0.010,SGD,0.372,0.327,0.174,0.268,0.110
5,yolov8,300,640,yolov8s,0.010,SGD,0.509,0.311,0.193,0.325,0.137
9,yolov8,60,800,yolov8s,0.010,SGD,0.581,0.293,0.195,0.318,0.125
10,yolov8,80,800,yolov8s,0.010,SGD,0.560,0.302,0.196,0.310,0.123
7,yolov8,100,800,yolov8s,0.010,SGD,0.578,0.302,0.198,0.314,0.124
1,yolov8,50,640,yolov8l,0.010,SGD,0.481,0.347,0.202,0.309,0.130


In [81]:
df[columns_of_interest].sort_values(by=['all_R']).rename(columns=new_column_names)

,Model,epochs,imgsz,TL,LR,Optim.,P,R,F1-score,mAP@.5,mAP@.5:.95
0,yolov8,50,640,yolov8l,0.001,Adam,0.352,0.276,0.155,0.210,0.080
4,yolov8,100,640,yolov8s,0.001,Adam,0.435,0.286,0.173,0.269,0.102
9,yolov8,60,800,yolov8s,0.010,SGD,0.581,0.293,0.195,0.318,0.125
6,yolov8,50,640,yolov8s,0.001,Adam,0.307,0.302,0.152,0.214,0.083
7,yolov8,100,800,yolov8s,0.010,SGD,0.578,0.302,0.198,0.314,0.124
10,yolov8,80,800,yolov8s,0.010,SGD,0.560,0.302,0.196,0.310,0.123
5,yolov8,300,640,yolov8s,0.010,SGD,0.509,0.311,0.193,0.325,0.137
8,yolov8,40,800,yolov8s,0.010,SGD,0.372,0.327,0.174,0.268,0.110
2,yolov8,100,640,yolov8m,0.010,SGD,0.576,0.335,0.212,0.366,0.145
3,yolov8,50,640,yolov8m,0.010,SGD,0.331,0.345,0.169,0.285,0.114


el 2

In [87]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v2_task_test_ds_rc.csv")

chosen = df[columns_of_interest].sort_values(by=['all_P']).rename(columns=new_column_names)
chosen

,Model,epochs,imgsz,TL,LR,Optim.,P,R,F1-score,mAP@.5,mAP@.5:.95
2,yolov8,50,640,yolov8m,0.010,SGD,0.331,0.345,0.169,0.285,0.114
0,yolov8,50,640,yolov8l,0.010,SGD,0.481,0.347,0.202,0.309,0.130
1,yolov8,100,640,yolov8m,0.010,SGD,0.576,0.335,0.212,0.366,0.145
3,yolov8,100,800,yolov8s,0.010,SGD,0.578,0.302,0.198,0.314,0.124
4,yolov8,60,800,yolov8s,0.010,SGD,0.581,0.293,0.195,0.318,0.125


In [88]:
short_caption = "Modelos elegidos para la optimización."

long_caption = """Modelos elegidos para la optimización."""

write_df_in_latex(chosen,"chosen",short_caption,long_caption)

\begin{table}[h]
	\centering
	\caption[Modelos elegidos para la optimización.]{Modelos elegidos para la optimización.}
	\tiny
\begin{tabular}{lrrlrlrrrrr}
\toprule
 Model &  epochs &  imgsz &      TL &    LR & Optim. &     P &     R &  F1-score &  mAP@.5 &  mAP@.5:.95 \\
\midrule
yolov8 &      50 &    640 & yolov8m & 0.010 &    SGD & 0.331 & 0.345 &     0.169 &   0.285 &       0.114 \\
yolov8 &      50 &    640 & yolov8l & 0.010 &    SGD & 0.481 & 0.347 &     0.202 &   0.309 &       0.130 \\
yolov8 &     100 &    640 & yolov8m & 0.010 &    SGD & 0.576 & 0.335 &     0.212 &   0.366 &       0.145 \\
yolov8 &     100 &    800 & yolov8s & 0.010 &    SGD & 0.578 & 0.302 &     0.198 &   0.314 &       0.124 \\
yolov8 &      60 &    800 & yolov8s & 0.010 &    SGD & 0.581 & 0.293 &     0.195 &   0.318 &       0.125 \\
\bottomrule
\end{tabular}

	\label{tab:chosen}
\end{table}
